In [16]:
!pip install eurostat pandas

# process_eurostat.py

# === Imports ===
import eurostat
import pandas as pd
import os

# Make sure output folder exists
os.makedirs("data", exist_ok=True)

print("Libraries loaded, folder ready.")


Libraries loaded, folder ready.


In [17]:
# === Step 1: Fetch data ===

# Hospital discharges by diagnosis (Eurostat dataset)
df_raw = eurostat.get_data_df('hlth_co_disch2')

print("Raw shape:", df_raw.shape)
print(df_raw.head())


Raw shape: (290813, 34)
  freq    age indic_he     unit sex  icd10 geo\TIME_PERIOD  1995  1996  1997  \
0    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z              AT   NaN   NaN   NaN   
1    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z              BE   NaN   NaN   NaN   
2    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z              BG   NaN   NaN   NaN   
3    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z              CH   NaN   NaN   NaN   
4    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z              CY   NaN   NaN   NaN   

   ...     2012     2013     2014     2015     2016     2017     2018  \
0  ...      NaN      NaN      NaN  26605.9      NaN      NaN      NaN   
1  ...  18216.1  17997.4  18121.6      NaN      NaN  17823.1  17885.6   
2  ...      NaN  33443.4  35209.0  35112.2  34533.6  35900.2  37174.4   
3  ...  17679.3  17747.7  17856.8  18006.7  18251.6  18023.4  17864.2   
4  ...      NaN   7451.0   7418.3   7322.8   7243.1   7406.3   7452.4   

      2019     2020     2021  
0      NaN      NaN      

In [18]:
# === Step 2: Reshape to long format ===
df = df_raw.melt(
    id_vars=['freq', 'age', 'indic_he', 'unit', 'sex', 'icd10', 'geo\TIME_PERIOD'],
    var_name='time',
    value_name='value'
)

# Rename geo column
df = df.rename(columns={'geo\TIME_PERIOD': 'geo'})

print("Reshaped shape:", df.shape)
print(df.head())


<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:9: SyntaxWarning: invalid escape sequence '\T'
<>:3: SyntaxWarning: invalid escape sequence '\T'
<>:9: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipython-input-782069064.py:3: SyntaxWarning: invalid escape sequence '\T'
  id_vars=['freq', 'age', 'indic_he', 'unit', 'sex', 'icd10', 'geo\TIME_PERIOD'],
/tmp/ipython-input-782069064.py:9: SyntaxWarning: invalid escape sequence '\T'
  df = df.rename(columns={'geo\TIME_PERIOD': 'geo'})


Reshaped shape: (7851951, 9)
  freq    age indic_he     unit sex  icd10 geo  time  value
0    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z  AT  1995    NaN
1    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z  BE  1995    NaN
2    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z  BG  1995    NaN
3    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z  CH  1995    NaN
4    A  TOTAL    INPAT  P_HTHAB   F  A-T_Z  CY  1995    NaN


In [19]:
# Drop rows with missing values
df = df.dropna(subset=['value']).copy()

# Convert year to int
df['time'] = df['time'].astype(int)

# Keep only records where age = TOTAL (all ages combined)
# and sex = T (total, both male and female combined).
# This simplifies the dataset so each row represents the full population,
# which is useful for high-level cross-country comparisons.
df = df[(df['age'] == 'TOTAL') & (df['sex'] == 'T')]

# Keep only hospital discharges unit if relevant
if 'unit' in df.columns:
    df = df[df['unit'] == 'P_HTHAB']   # adapt depending on meaning


In [20]:
# Filter to last 10 years (2012–2021)
df = df[df['time'] >= 2017]

print("Filtered shape:", df.shape)
print(df[['geo','icd10','time','value']].head(10))


Filtered shape: (20837, 9)
        geo  icd10  time    value
6407854  BE  A-T_Z  2017  16792.1
6407855  BG  A-T_Z  2017  32946.3
6407856  CH  A-T_Z  2017  17092.8
6407857  CY  A-T_Z  2017   7751.5
6407858  CZ  A-T_Z  2017  19813.6
6407859  DE  A-T_Z  2017  25477.9
6407861  ES  A-T_Z  2017  10435.7
6407862  FI  A-T_Z  2017  16423.5
6407863  FR  A-T_Z  2017  18608.9
6407864  HR  A-T_Z  2017  16022.7


In [21]:
# Save cleaned file
df.to_csv("data/hospital_discharges_clean.csv", index=False)

# Save latest year only
latest_year = df['time'].max()
df_latest = df[df['time'] == latest_year].copy()
df_latest.to_csv("data/hospital_discharges_latest.csv", index=False)

print(f"Saved processed files. Latest year = {latest_year}")


Saved processed files. Latest year = 2021
